## GFDx Notebook: Food Fortification Opportunity Algorithm SPANISH



---




In [ ]:
# Install package to allow import from REDCap API
from redcap import Project
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm  # progress bar

# Connecting to GFDx Redcap API

api_key = os.environ.get("APIKEY")

# Connecting to GFDx Redcap API
URL = 'https://redcap.emory.edu/api/'
project = Project(URL, api_key)

In [ ]:
# Pulls out variables of interest from REDCap in table form
fields_of_interest = ['country_code', 'country_territory', 'food_intake', 'import_pc', 'coverage_fv', 'industrially_processed_pc', 
                      'coverage_ipfv', 'status_food', 'legislation_scope_types', 'legislation_scope_origins', 'legislation_scope_origins___1', 'legislation_scope_origins___2', 'legislation_scope_origins___3', 
                      'legislation_scope_uses', 'legislation_scope_uses___1', 'legislation_scope_uses___2', 'legislation_scope_uses___3', 'legislation_scope_uses___4', 'imp_applicability', 
                      'fortification_standard', 'ext_mon_protocol', 'imp_mon_protocol']
subset = project.export_records(fields=fields_of_interest, format='df')

In [ ]:
# Reset index
df = subset.copy()
df.reset_index(inplace=True)

In [ ]:
# Create list of unique food vehicles
food_list = ['maize_flour_arm_2', 'wheat_flour_arm_2', 'rice_arm_2', 'salt_arm_2', 'oil_arm_2']

# Territory codes that need to be removed
territory_codes = df[df.country_territory == 1].country_code.unique()

In [ ]:
df = df[(df.country_code.isin(territory_codes)) & (df.country_code != 999.0)]

In [ ]:
# Filtered for data in the food_list
filt_df = df[df.redcap_event_name.isin(food_list)]
countries = filt_df.country_code.unique()  # Unique country codes from the filtered data
foods = filt_df.redcap_event_name.unique() # Unique food vehicles from filtered data

In [ ]:
# List of all columns with repeating forms used in this analysis
targets = [
           {'instrument': 'intake', 'col': ['food_intake', 'import_pc']}, 
           {'instrument': 'coverage_fv', 'col': 'coverage_fv'},
           {'instrument': 'industrially_processed', 'col': 'industrially_processed_pc'}, 
           {'instrument': 'coverage_ipfv', 'col': 'coverage_ipfv' },
           ]

In [ ]:
# Pulls the latest year of each instrument in targets and saves into "new_df"
new_df = pd.DataFrame()
for country in tqdm(countries):
    for food in foods:
        for target in targets:
            sliced = filt_df[
                             (filt_df.country_code == country) & 
                             (filt_df.redcap_event_name == food) & 
                             (filt_df.redcap_repeat_instrument == target['instrument'])
                             ]
            if target['instrument'] == 'intake':
                single = sliced[sliced.redcap_repeat_instance == sliced.redcap_repeat_instance.max()][['country_code', 'redcap_event_name', 'redcap_repeat_instrument', 'food_intake', 'import_pc']]
                if len(single) == 0:
                    single_revised = pd.DataFrame(
                        [[country, food, target['instrument'], np.NaN, np.NaN]], 
                        columns=['country_code', 'redcap_event_name', 'redcap_repeat_instrument', 'food_intake', 'import_pc']
                    )
                    new_df = new_df.append(single_revised, ignore_index=True)
                else:
                    new_df = new_df.append(single, ignore_index=True)
            else:
                single = sliced[sliced.redcap_repeat_instance == sliced.redcap_repeat_instance.max()][['country_code', 'redcap_event_name', 'redcap_repeat_instrument', target['col']]]
                if len(single) == 0:
                    single_revised =  pd.DataFrame(
                        [[country, food, target['instrument'], np.NaN]], 
                        columns=['country_code', 'redcap_event_name', 'redcap_repeat_instrument', target['col']]
                    )
                    new_df = new_df.append(single_revised, ignore_index=True)
                else:
                    new_df = new_df.append(single, ignore_index=True)


In [ ]:
# Saves the arms into variables that will be used later
grains = ['maize_flour_arm_2', 'wheat_flour_arm_2', 'rice_arm_2']
oil = 'oil_arm_2'
salt = 'salt_arm_2'

In [ ]:
# To use for food vehicle "fill-in" in later formulas
def lowercase(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "la harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "la harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "el arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "la sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "el aceite"

new_df['food_lower'] = new_df.apply(lambda row: lowercase(row), axis=1)

# To use for food vehicle "fill-in" in later formulas
def lowercasefood(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "aceite"

new_df['food_lower_food'] = new_df.apply(lambda row: lowercasefood(row), axis=1)

# To use for food vehicle "fill-in" in later formulas
def oflowercase(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite"

new_df['of_food_lower'] = new_df.apply(lambda row: oflowercase(row), axis=1)

# To use for food vehicle "fill-in" in later formulas
def oflowercaseproduce(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz producida"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo producida"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz producido"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal producida"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite producido"

new_df['of_food_lower_produce'] = new_df.apply(lambda row: oflowercaseproduce(row), axis=1)

In [ ]:
# Step 1 - categorize avaiablilty/intake
# Create formula
def identify_availability(row):
    return_val = ''
    if row.redcap_event_name in grains:
        if row.food_intake < 75:
            return_val = 'Ingesta/disponibilidad baja'
        elif row.food_intake < 150:
            return_val = 'Ingesta/disponibilidad moderada '
        elif row.food_intake < 300:
            return_val = 'Ingesta/disponibilidad moderadamente alta'
        elif row.food_intake >= 300:
            return_val = 'Ingesta/disponibilidad alta'
        else:
            return_val = 'Ingesta/disponibilidad desconocida'
    elif row.redcap_event_name == oil:
        if row.food_intake < 20:
            return_val = 'Ingesta/disponibilidad baja'
        elif row.food_intake < 40:
            return_val = 'Ingesta/disponibilidad moderada '
        elif row.food_intake < 60:
            return_val = 'Ingesta/disponibilidad moderadamente alta'
        elif row.food_intake >= 60:
            return_val = 'Ingesta/disponibilidad alta'
        else:
            return_val = 'Ingesta/disponibilidad desconocida' 
    elif row.redcap_event_name == salt:
        return_val = 'Ingesta/disponibilidad alta'
    else:
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val

# Step 1 - categorize coverage
# Create formula

def identify_coverage(row):
    return_val = ''
    if row.redcap_event_name in grains:
        if row.coverage_fv < 35:
            return_val = 'Cobertura de la población baja'
        elif row.coverage_fv < 75:
            return_val = 'Cobertura de la población moderada'
        elif row.coverage_fv >= 75:
            return_val = 'Cobertura de la población alta'
        else:
            return_val = 'Cobertura de la población desconocida'
    elif row.redcap_event_name == oil:
        if row.coverage_fv < 35:
            return_val = 'Cobertura de la población baja'
        elif row.coverage_fv < 75:
            return_val = 'Cobertura de la población moderada'
        else:
            return_val = 'Cobertura de la población alta'
    elif row.redcap_event_name == salt:
            return_val = 'Cobertura de la población alta'
    else:
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val

In [ ]:
# Applies both the formulas above and adds two new columns with the result
new_df['food_avail'] = new_df.apply(lambda row: identify_availability(row), axis=1)
new_df['pop_coverage'] = new_df.apply(lambda row: identify_coverage(row), axis=1)

In [ ]:
# Step 1 - Comparing availability/intake and coverage to assess reach
# Create formula

def assess_reach(x, y):
    return_val = ''
    if x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población baja':
        return_val = 'Bajo'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población moderada':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población alta':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población baja':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población moderada':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población alta':
        return_val = 'Alto'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población baja':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población moderada':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población alta':
        return_val = 'Alto'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población baja':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población moderada':
        return_val = 'Alto'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población alta':
        return_val = 'Alto'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población desconocida':
        return_val = 'Se necesitan más datos'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población baja':
        return_val = 'Bajo'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población moderada':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población alta':
        return_val = 'Alto'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población desconocida':
        return_val = 'Bajo'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población desconocida':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población desconocida':
        return_val = 'Moderado'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población desconocida':
        return_val = 'Alto'
    return return_val


In [ ]:
def assess_reach_comment(x, y, t, w):
    return_val = ''
    if x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población baja':
        return_val = f'El beneficio de fortificar {t} es bajo.'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población moderada':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población alta':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población baja':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población moderada':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población alta':
        return_val = f'El beneficio de fortificar {t} es alto.'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población baja':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población moderada':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población alta':
        return_val = f'El beneficio de fortificar {t} es alto.'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población baja':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población moderada':
        return_val = f'El beneficio de fortificar {t} es alto.'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población alta':
        return_val = f'El beneficio de fortificar {t} es alto.'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población desconocida':
        return_val = f'Se necesitan más datos sobre los patrones de consumo {w} para saber si la fortificación beneficiaría a la población.'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población baja':
        return_val = f'El beneficio de fortificar {t} es bajo.'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población moderada':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad desconocida' and y == 'Cobertura de la población alta':
        return_val = f'El beneficio de fortificar {t} es alto.'
    elif x == 'Ingesta/disponibilidad baja' and y == 'Cobertura de la población desconocida':
        return_val = f'El beneficio de fortificar {t} es bajo.'
    elif x == 'Ingesta/disponibilidad moderada' and y == 'Cobertura de la población desconocida':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad moderadamente alta' and y == 'Cobertura de la población desconocida':
        return_val = f'El beneficio de fortificar {t} es moderado.'
    elif x == 'Ingesta/disponibilidad alta' and y == 'Cobertura de la población desconocida':
        return_val = f'El beneficio de fortificar {t} es alto.'
    return return_val

In [ ]:
# Step 2 - Categorizing industrially processed
# Create formula
def assess_industry(df):
    salt_event = df[df.redcap_repeat_instrument == 'industrially_processed'].redcap_event_name.iloc[0]
    industrial_var = df[df.redcap_repeat_instrument == 'industrially_processed'].industrially_processed_pc.iloc[0]
    coverage_var = df[df.redcap_repeat_instrument == 'coverage_ipfv'].coverage_ipfv.iloc[0]
    import_var = df[df.redcap_repeat_instrument == 'intake'].import_pc.iloc[0]
    if salt_event == 'salt_arm_2':
        return 'Alto'
    elif industrial_var < 35:
        return 'Bajo'
    elif industrial_var < 75:
        return 'Moderado'
    elif industrial_var >= 75:
        return 'Alto'
    elif coverage_var < 35:
        return 'Bajo'
    elif coverage_var < 75:
        return 'Moderado'
    elif coverage_var >= 75:
        return 'Alto'
    elif import_var < 35:
        return 'Bajo'
    elif import_var < 75:
        return 'Moderado'
    elif import_var >= 75:
        return 'Alto'
    else:
        return 'Se necesitan más datos'

In [ ]:
def assess_industry_comment(df, w):
    salt_event = df[df.redcap_repeat_instrument == 'industrially_processed'].redcap_event_name.iloc[0]
    industrial_var = df[df.redcap_repeat_instrument == 'industrially_processed'].industrially_processed_pc.iloc[0]
    coverage_var = df[df.redcap_repeat_instrument == 'coverage_ipfv'].coverage_ipfv.iloc[0]
    import_var = df[df.redcap_repeat_instrument == 'intake'].import_pc.iloc[0]
    if salt_event == 'salt_arm_2':
        return 'Una proporción alta de la sal se procesa industrialmente; lo cual sugiere que la fortificación de la sal se puede implementar de manera costo-efectiva y sotenible.'
    elif industrial_var < 35:
        return f'Una proporción baja {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible.'
    elif industrial_var < 75:
        return f'Una proporción moderada {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible para la proporción de alimento que no se procesa industrialmente.'
    elif industrial_var >= 75:
        return f'Una proporción alta {w} se procesa industrialmente; lo cual sugiere que la fortificación {w} se puede implementar de manera costo-efectiva y sotenible.'
    elif coverage_var < 35:
        return f'Una proporción baja {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible.'
    elif coverage_var < 75:
        return f'Una proporción moderada {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible para la proporción de alimento que no se procesa industrialmente.'
    elif coverage_var >= 75:
        return f'Una proporción alta {w} se procesa industrialmente; lo cual sugiere que la fortificación {w} se puede implementar de manera costo-efectiva y sotenible.'
    elif import_var < 35:
            return f'Una proporción baja {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible.'
    elif import_var < 75:
        return f'Una proporción moderada {w} se procesa industrialmente; la fortificación {w} puede no ser costo-efectiva ni sotenible para la proporción de alimento que no se procesa industrialmente.'
    elif import_var >= 75:
        return f'Una proporción alta {w} se procesa industrialmente; lo cual sugiere que la fortificación {w} se puede implementar de manera costo-efectiva y sotenible.'
    else:
        return f'Se necesita información sobre el panorama de la industria (proporción {w} molida industrialmente) para saber si la fortificación {w} se podría implementar de manera costo-efectiva y sostenible.'

In [ ]:
# This applies the above formulas and gets a list of the results for each
# These lists are used later as columns in the table

new_countries = []
new_foods = []
food_availabilities = []
industry_processes = []
industry_processes_comment = []
pop_coverages = []
reaches = []
reaches_comment = []
for country in new_df.country_code.unique():
    for food in new_df.redcap_event_name.unique():
            temp = new_df[
                            (new_df.country_code == country) & 
                            (new_df.redcap_event_name == food)
                            ]

            food_avail = temp[temp.redcap_repeat_instrument == 'intake'].food_avail.iloc[0]
            pop_cover = temp[temp.redcap_repeat_instrument == 'coverage_fv'].pop_coverage.iloc[0]
            lower_val = temp.food_lower.values[0]
            lower_val_food = temp.food_lower_food.values[0]
            of_lower_val = temp.of_food_lower.values[0]

            new_countries.append(country)
            new_foods.append(food)

            food_availabilities.append(food_avail)
            pop_coverages.append(pop_cover)

            industry_processes.append(assess_industry(temp))
            industry_processes_comment.append(assess_industry_comment(temp, of_lower_val))
            reaches.append(assess_reach(food_avail, pop_cover))
            reaches_comment.append(assess_reach_comment(food_avail, pop_cover, lower_val_food, of_lower_val))

In [ ]:
# Step 3 - Comparing reach potential (step 1) and industrially processed (step 2)
# Create formula

def assess_overall(row):
    if row.reach_potential == 'Alto' and row.operational_ease_cat == 'Alto':
        return 'Bueno'
    elif row.reach_potential == 'Alto' and row.operational_ease_cat == 'Moderado':
        return 'Bueno'
    elif row.reach_potential == 'Alto' and row.operational_ease_cat == 'Bajo':
        return 'Moderado'
    elif row.reach_potential == 'Alto' and row.operational_ease_cat == 'Se necesitan más datos':
        return 'Moderado'
    elif row.reach_potential == 'Moderado' and row.operational_ease_cat == 'Alto':
        return 'Bueno'
    elif row.reach_potential == 'Moderado' and row.operational_ease_cat == 'Moderado':
        return 'Bueno'
    elif row.reach_potential == 'Moderado' and row.operational_ease_cat == 'Bajo':
        return 'Moderado'
    elif row.reach_potential == 'Moderado' and row.operational_ease_cat == 'Se necesitan más datos':
        return 'Moderado'
    elif row.reach_potential == 'Bajo' and row.operational_ease_cat == 'Alto':
        return 'Moderado'
    elif row.reach_potential == 'Bajo' and row.operational_ease_cat == 'Moderado':
        return 'Moderado'
    elif row.reach_potential == 'Bajo' and row.operational_ease_cat == 'Bajo':
        return 'Pobre'
    elif row.reach_potential == 'Bajo' and row.operational_ease_cat == 'Se necesitan más datos':
        return 'Pobre'
    elif row.reach_potential == 'Se necesitan más datos' and row.operational_ease_cat == 'Alto':
        return 'Moderado'
    elif row.reach_potential == 'Se necesitan más datos' and row.operational_ease_cat == 'Moderado':
        return 'Moderado'
    elif row.reach_potential == 'Se necesitan más datos' and row.operational_ease_cat == 'Bajo':
        return 'Pobre'
    elif row.reach_potential == 'Se necesitan más datos' and row.operational_ease_cat == 'Se necesitan más datos':
        return 'Se necesitan más datos'


In [ ]:
# Step 3 - Comparing reach potential (step 1) and industrially processed (step 2)
# Create formula
# Full comment

def assess_overall_comment(row):
    if row.reach_potential == 'High' and row.operational_ease_cat == 'High':
        return f'La fortificación {row.of_food_lower} es una buena oportunidad para tener un impacto sobre la salud a nivel nacional.'
    elif row.reach_potential == 'High' and row.operational_ease_cat == 'Moderate':
        return f'La fortificación {row.of_food_lower} es una buena oportunidad para tener un impacto a nivel nacional. Una mayor proporción {row.of_food_lower_produce} industrialmente fortalecería esta oportunidad.'
    elif row.reach_potential == 'High' and row.operational_ease_cat == 'Low':
        return f'La fortificación {row.of_food_lower} es una oportunidad moderada para tener un impacto sobre la salud a nivel nacional. Una proporción significativamente mayor {row.of_food_lower_produce} industrialmente mejoraría esta oportunidad.'
    elif row.reach_potential == 'High' and row.operational_ease_cat == 'More data needed':
        return f'La fortificación {row.of_food_lower} es una oportunidad moderada para tener un impacto a nivel nacional. Se necesita más información sobre la proporción {row.of_food_lower_produce} industrialmente para confirmar.'
    elif row.reach_potential == 'Moderate' and row.operational_ease_cat == 'High':
        return f'La fortificación {row.of_food_lower} es una buena oportunidad para tener un impacto sobre la salud a nivel nacional, en particular si hay un adecuado consumo y cobertura {row.of_food_lower} en poblaciones con una  dieta de mala calidad.'
    elif row.reach_potential == 'Moderate' and row.operational_ease_cat == 'Moderate':
        return f'La fortificación {row.of_food_lower} es una buena oportunidad para tener un impacto sobre la salud a nivel nacional, en particular si hay un adecuado consumo y cobertura {row.of_food_lower} en poblaciones con una  dieta de mala calidad. Una mayor proporción {row.of_food_lower_produce} industrialmente fortalecería esta oportunidad.'
    elif row.reach_potential == 'Moderate' and row.operational_ease_cat == 'Low':
        return f'La fortificación {row.of_food_lower} es una oportunidad moderada, en particular si hay un adecuado consumo y cobertura {row.of_food_lower} en poblaciones con una dieta de mala calidad. Una proporción significativamente mayor {row.of_food_lower_produce} industrialmente mejoraría esta oportunidad.'
    elif row.reach_potential == 'Moderate' and row.operational_ease_cat == 'More data needed':
        return f'La fortificación {row.of_food_lower} es una oportunidad moderada, en particular si hay un adecuado consumo y cobertura {row.of_food_lower} en poblaciones con una dieta de mala calidad. Se necesita más información sobre la proporción {row.of_food_lower_produce} industrialmente para confirmar.'
    elif row.reach_potential == 'Low' and row.operational_ease_cat == 'High':
        return f'La fortificación {row.of_food_lower} es una oportunidad de fortificación moderada para tener un impacto a nivel nacional. Considerar la fortificación de un alimento diferente u otro alimento adicional a {row.food_lower}.'
    elif row.reach_potential == 'Low' and row.operational_ease_cat == 'Moderate':
        return f'La fortificación {row.of_food_lower} es una oportunidad de fortificación moderada para tener un impacto a nivel nacional. Considerar si otros alimentos son convenientes para la fortificación. Una mayor proporción {row.of_food_lower_produce} industrialmente fortalecería esta oportunidad.'
    elif row.reach_potential == 'Low' and row.operational_ease_cat == 'Low':
        return f'La fortificación {row.of_food_lower} es una oportunidad de fortificación deficiente para tener un impacto a nivel nacional. Para aumentar el impacto, considerar si otros alimentos son convenientes para la fortificación e intervenciones complementarias de micronutrientes. Continuar monitoreando la situación ya que el alcance potencial y la proporción de producido industrialmente puede cambiar con el tiempo.'
    elif row.reach_potential == 'Low' and row.operational_ease_cat == 'More data needed':
        return f'La fortificación {row.of_food_lower} es una oportunidad de fortificación deficiente para tener un impacto a nivel nacional. Para aumentar el impacto, considerar si otros alimentos son convenientes para la fortificación e intervenciones complementarias de micronutrientes. Se necesita más información sobre la proporción {row.of_food_lower_produce} industrialmente para confirmar.'
    elif row.reach_potential == 'More data needed' and row.operational_ease_cat == 'High':
        return f'La oportunidad de fortificación es poco clara para tener un impacto a nivel nacional. Para aumentar el impacto, considerar si otros alimentos son convenientes para la fortificación e intervenciones complementarias de micronutrientes. Se necesita más información sobre la ingesta/disponibilidad y cobertura {row.of_food_lower} para confirmar.'
    elif row.reach_potential == 'More data needed' and row.operational_ease_cat == 'Moderate':
        return f'La oportunidad de fortificación es poco clara para tener un impacto a nivel nacional. Para aumentar el impacto, considerar si otros alimentos son convenientes para la fortificación e intervenciones complementarias de micronutrientes. Se necesita más información sobre la ingesta/disponibilidad y cobertura {row.of_food_lower} para confirmar.'
    elif row.reach_potential == 'More data needed' and row.operational_ease_cat == 'Low':
        return f'La oportunidad de fortificación es deficiente para tener un impacto a nivel nacional. Para aumentar el impacto, considerar si otros alimentos son convenientes para la fortificación e intervenciones complementarias de micronutrientes. Se necesita más información sobre la ingesta/disponibilidad {row.of_food_lower} y cobertura en la población.'
    elif row.reach_potential == 'More data needed' and row.operational_ease_cat == 'More data needed':
        return f'La oportunidad de fortificación es desconocida. Se necesita más información sobre la ingesta/disponibilidad {row.of_food_lower}, cobertura y la proporción de producido industrialmente para indentificar oportunidades de fortificación.'


In [ ]:
# Lowercase food values for next dataframe

# To use for food vehicle "fill-in" in later formulas
def lowercase(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "la harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "la harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "el arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "la sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "el aceite"

# To use for food vehicle "fill-in" in later formulas
def lowercasefood(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "aceite"

# To use for food vehicle "fill-in" in later formulas
def oflowercase(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite"

# To use for food vehicle "fill-in" in later formulas
def oflowercasefortify(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz fortificada"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo fortificada"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz fortificado"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal fortificada"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite fortificado"

# To use for food vehicle "fill-in" in later formulas
def oflowercaseuse(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz utilizada"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo utilizada"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz utilizado"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal utilizada"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite utilizado"

# To use for food vehicle "fill-in" in later formulas
def oflowercasedonate(row):
    if row.redcap_event_name == 'maize_flour_arm_2':
        return "de la harina de maíz donada"
    elif row.redcap_event_name == 'wheat_flour_arm_2':
        return "de la harina de trigo donada"
    elif row.redcap_event_name == 'rice_arm_2':
        return "del arroz donado"
    elif row.redcap_event_name == 'salt_arm_2':
        return "de la sal donada"
    elif row.redcap_event_name == 'oil_arm_2':
        return "del aceite donado"

In [ ]:
# Create first dataset with only the result of the analysis
# Applies the respective formulas for each column
# Includes list results from above

final = pd.DataFrame()
final['country_code'] = [int(c) for c in new_countries]
final['redcap_event_name'] = new_foods
final['food_lower'] = final.apply(lambda row: lowercase(row), axis=1)
final['food_lower_food'] = final.apply(lambda row: lowercasefood(row), axis=1)
final['of_food_lower'] = final.apply(lambda row: oflowercase(row), axis=1)
final['of_food_lower_fortify'] = final.apply(lambda row: oflowercasefortify(row), axis=1)
final['of_food_lower_use'] = final.apply(lambda row: oflowercaseuse(row), axis=1)
final['of_food_lower_donate'] = final.apply(lambda row: oflowercasedonate(row), axis=1)
final['intake_cat'] = food_availabilities
final['coverage_fv_cat'] = pop_coverages
final['reach_potential'] = reaches
final['reach_potential_comment'] = reaches_comment
final['reach_potential_supp'] = final.reach_potential.apply(lambda x: 'Sí' if x == 'Moderado' or x == 'Bajo' else 'No')
final['operational_ease_cat'] = industry_processes
final['operational_ease_cat_comment'] = industry_processes_comment
final['operational_ease_supp'] = final.operational_ease_cat.apply(lambda x: 'Sí' if x == 'Moderado' or x == 'Bajo' else 'No')
final['overall_recommendation'] = final.apply(lambda row: assess_overall(row), axis=1)
final['overall_rec_comment'] = final.apply(lambda row: assess_overall_comment(row), axis=1)
final['overall_rec_supp'] = final.overall_recommendation.apply(lambda x: 'Sí' if x == 'Moderado' or x == 'Bajo' else 'No')


In [ ]:
# Step 4 - Comparing legislation status with overall recommendation (step 3) and legislation scope
# Pull data from original database and add to current final table

legislative_status = []
legislation_scope_types = []
legislation_scope_origins___1 = []
legislation_scope_origins___2 = []
legislation_scope_origins___3 = []
legislation_scope_uses___1 = []
legislation_scope_uses___2 = []
legislation_scope_uses___3 = []
legislation_scope_uses___4 = []
imp_applicability = []
standard = []
ext_protocol = []
imp_protocol = []


for country in filt_df.country_code.unique():
    for food in filt_df.redcap_event_name.unique():
        small = filt_df[(filt_df.country_code == country) & (filt_df.redcap_event_name == food)]

        if len(small) == 0:
            print(country, food)
            continue
        legislative_status.append(small.status_food.iloc[0])
        legislation_scope_types.append(small.legislation_scope_types.iloc[0])
        legislation_scope_origins___1.append(small.legislation_scope_origins___1.iloc[0])
        legislation_scope_origins___2.append(small.legislation_scope_origins___2.iloc[0])
        legislation_scope_origins___3.append(small.legislation_scope_origins___3.iloc[0])
        legislation_scope_uses___1.append(small.legislation_scope_uses___1.iloc[0])
        legislation_scope_uses___2.append(small.legislation_scope_uses___2.iloc[0])
        legislation_scope_uses___3.append(small.legislation_scope_uses___3.iloc[0])
        legislation_scope_uses___4.append(small.legislation_scope_uses___4.iloc[0])
        imp_applicability.append(small.imp_applicability.iloc[0])
        standard.append(small.fortification_standard.iloc[0])
        ext_protocol.append(small.ext_mon_protocol.iloc[0])
        imp_protocol.append(small.imp_mon_protocol.iloc[0])


final['status_food'] = legislative_status
final['fortification_standard'] = standard
final['ext_mon_protocol'] = ext_protocol
final['imp_mon_protocol'] = imp_protocol
final['legislation_scope_types'] = legislation_scope_types
final['domestically_produced'] = legislation_scope_origins___1
final['imports'] = legislation_scope_origins___2
final['exports'] = legislation_scope_origins___3
final['household'] = legislation_scope_uses___1
final['processed_food'] = legislation_scope_uses___2
final['animal_feed'] = legislation_scope_uses___3
final['donated_food'] = legislation_scope_uses___4
final['imp_applicability'] = imp_applicability

In [ ]:
# Step 4 - Comparing legislation status with overall recommendation (step 3)
# Create formula

def status_comparison(row):
    if row.status_food == 1 and row.overall_recommendation == 'Bueno':
        return f'La fortificación {row.of_food_lower} ya es obligatoria; verificar la proporción de alimento que se está fortificando (datos de cumplimiento/calidad) si el programa se está implementando óptimamente, se podría esperar un impacto a nivel nacional.'
    elif row.status_food == 1 and row.overall_recommendation == 'Moderado':
        return f'La fortificación {row.of_food_lower} ya es obligatoria; verificar la proporción de alimento que se está fortificando (datos de cumplimiento/calidad) si el programa se está implementando óptimamente, se podría esperar un impacto a nivel nacional o en sub-grupos.'
    elif row.status_food == 1 and row.overall_recommendation == 'Pobre':
        return f'La fortificación {row.of_food_lower} ya es obligatoria.'
    elif row.status_food == 1 and row.overall_recommendation == 'Se necesitan más datos':
        return f'La fortificación {row.of_food_lower} ya es obligatoria.'
    elif row.status_food == 2 and row.overall_recommendation == 'Bueno':
        return f'La fortificación {row.of_food_lower} es voluntaria; considerar si se puede lograr un mayor impacto si la fortificación fuese obligatoria.'
    elif row.status_food == 2 and row.overall_recommendation == 'Moderado':
        return f'La fortificación {row.of_food_lower} es voluntaria; considerar si se puede lograr un mayor impacto si la fortificación fuese obligatoria.'
    elif row.status_food == 2 and row.overall_recommendation == 'Pobre':
        return f'La fortificación {row.of_food_lower} es voluntaria; considerar la fortificación obligatoria de otros alimentos que tienen una buena oportunidad para la fortificación.'
    elif row.status_food == 2 and row.overall_recommendation == 'Se necesitan más datos':
        return f'La fortificación {row.of_food_lower} es voluntaria; considerar si se puede lograr un mayor impacto si la fortificación fuese obligatoria y considerar estrategias para fortalecer la oportunidad cuando haya más información disponible sobre la oportunidad de fortificación.'
    elif row.status_food == 3 and row.overall_recommendation == 'Bueno':
        return f'Existe una buena oportunidad de que la fortificación tenga un impacto nutricional a nivel nacional; si ésta no es obligatoria, considerar reunirse con las partes interesadas para discutir la oportunidad de adoptar la fortificación obligatoria {row.of_food_lower}.'
    elif row.status_food == 3 and row.overall_recommendation == 'Moderado':
        return f'Existe una oportunidad moderada de que la fortificación tenga un impacto nutricional a nivel nacional; considerar reunir a las partes interesadas clave dentro de los reguladores gubernamentales y la industria para discutir a fondo.'
    elif row.status_food == 3 and row.overall_recommendation == 'Pobre':
        return f'La fortificación puede no ser la mejor intervención para mejorar la nutrición a nivel nacional; considerar reunir a las partes interesadas clave dentro de los reguladores gubernamentales y la industria para discutir a fondo. Es probable que intervenciones complementarias serán necesarias en lugar de o en conjunto con la fortificación para abordar los desafios nutricionales.'
    elif row.status_food == 3 and row.overall_recommendation == 'Se necesitan más datos':
        return f'No hay suficientes datos para determinar si la fortificación podría proveer una buena oportunidad para mejorar la nutrición. Considerar recopilar información adicional para evaluar la situación a fondo.'
    else:
        return 'Sin evaluación'

In [ ]:
# Step 4 - Comparing legislation status with Presence of fortification standards
# Create formula

def standard_comparison(row):
    if row.status_food == 1 and row.fortification_standard == 2:
        return 'Considerar desarrollar estándares para este alimento para proveer orientación a la industria sobre las especificaciones en la calidad y nutrientes que tienen que cumplir sus productos.'
    elif row.status_food == 1 and row.fortification_standard == 3:
        return f'El Intercambio de Datos (GFDx, siglas en inglés) no tiene los estándares de fortificación para {row.food_lower}.'
    elif row.status_food == 2 and row.fortification_standard == 2:
        return 'Considerar desarrollar estándares para este alimento para proveer orientación a la industria sobre las especificaciones en la calidad y nutrientes que tienen que cumplir sus productos.'    
    elif row.status_food == 2 and row.fortification_standard == 3:
        return f'El Intercambio de Datos (GFDx, siglas en inglés) no tiene los estándares de fortificación para {row.food_lower}.'
    elif row.status_food == 3 and row.fortification_standard == 1:
        return f'El Intercambio de Datos (GFDx, siglas en inglés) no tiene documentación sobre si la fortificación {row.of_food_lower} es obligatoria o voluntaria.'
    else:
        return 'Sin evaluación'

In [ ]:
# Step 4 - Comparing legislation status with Presence of import monitoring protocols
# Create formula

def import_comparison(row):
    if row.status_food == 1 and row.imp_mon_protocol == 1:
        return 'Existen protocolos de monitoreo de las importaciones, lo cual debe indicar claridad sobre las funciones y responsabilidades de las partes interesadas, la frecuencia y procedimientos de monitoreo, y la presentación de informes e intercambio de datos con la industria. Considerar la revisión de estos protocolos y realizar mejoras en ellos si es necesario.'
    elif row.status_food == 1 and row.imp_mon_protocol == 2:
        return 'Conderar el desarrollo de protocolos de monitoreo de las importaciones para este alimento para definir las funciones y responsabilidades de las partes interesadas, la frecuencia y procedimientos de monitoreo, y la presentación de informes e intercambio de datos con la industria.'
    elif row.status_food == 1 and row.imp_mon_protocol == 3:
        return f'El Intercambio de Datos (GFDx, siglas en inglés) no tiene el protocolo de monitoreo de las importaciones {row.of_food_lower}.'
    else:
        return 'Este alimento no se importa.'


In [ ]:
# Step 4 - Comparing legislation status with Presence of export monitoring protocols
# Create formula

def export_comparison(row):
    if row.status_food == 1 and row.ext_mon_protocol == 1:
        return 'Existen protocolos de monitoreo de las exportaciones, lo cual debe indicar claridad sobre las funciones y responsabilidades de las partes interesadas, la frecuencia y procedimientos de monitoreo, y la presentación de informes e intercambio de datos con la industria. Considerar la revisión de estos protocolos y realizar mejoras en ellos si es necesario.'
    elif row.status_food == 1 and row.ext_mon_protocol == 2:
        return 'Conderar el desarrollo de protocolos de monitoreo de las exportaciones para este alimento para definir las funciones y responsabilidades de las partes interesadas, la frecuencia y procedimientos de monitoreo, y la presentación de informes e intercambio de datos con la industria.'
    elif row.status_food == 1 and row.ext_mon_protocol == 3:
        return f'El Intercambio de Datos (GFDx, siglas en inglés) no tiene el protocolo de monitoreo de las exportaciones {row.of_food_lower}.'
    else:
        return 'Este alimento no se produce en el país.'

In [ ]:
# Step 4 - Comparing legislation status with legislation scope
# Create formula

def scope_type(row):
    if row.status_food == 1 and row.legislation_scope_types == 1:
        return f'Se requiere la fortificación para todos los tipos {row.of_food_lower}.'
    elif row.status_food == 1 and row.legislation_scope_types == 2:
        return f'La fortificación solo se requiere para un subconjunto de tipos {row.of_food_lower} types. Revisar los patrones de consumo periodicamente para asegurarse que se requiere la fortificación para los tipos de alimentos de mayor consumo.'
    else:
        return 'Sin evaluación'

def origin_dp(row):
    if row.status_food == 1 and row.domestically_produced == 1 and row.imp_applicability == 1:
        return f'Se requiere la fortificación para {row.food_lower} que se producen en el país (cuando sea relevante).'
    elif row.status_food == 1 and row.domestically_produced == 0:
        return f'No se requiere la fortificación para todo {row.food_lower} que se producen en el país (cuando sea relevante). Si {row.food_lower} que se produce en el país se procesa industrialmente, considerar la posibilidad de enmendar la legislación y/o las normas para incluir alimentos de producción nacional.'
    else:
        return 'Sin evaluación'

def origin_imports(row):
    if row.status_food == 1 and row.imports == 1 and row.imp_applicability == 1:
        return f'Se requiere la fortificación para {row.food_lower} que se importan (cuando sea relevante).'
    elif row.status_food == 1 and row.imports == 0:
        return f'No se requiere la fortificación para todo {row.food_lower} que se importan (cuando sea relevante). Si se requiere la fortificación para {row.food_lower} que se produce en el país, se debe enmendar la legislación y/o las normas para aplicar los mismos requisitos a los alimentos importados.'
    else:
        return 'Sin evaluación'

def origin_exports(row):
    if row.status_food == 1 and row.exports == 1:
        return f'La legislación nacional generalmente no incluye {row.food_lower_food} que se exportan, ya que se espera que cumplan con las normas de los países importadores. Considerar si el requisito de fortificar los alimentos de exportación coloca a la industria en una posición dificil de cumplir.'
    elif row.status_food == 1 and row.exports == 0:
        return f'La legislación nacional generalmente no incluye {row.food_lower_food} que se exportan, ya que se espera que cumplan con las normas de los países importadores.'
    else:
        return 'Sin evaluación'

def uses_household(row):
    if row.status_food == 1 and row.household == 1:
        return f'Se requiere la fortificación para {row.food_lower} que se destina para el uso doméstico, lo cual es una de las formas clave que los consumidores consumen {row.food_lower_food}.'
    elif row.status_food == 1 and row.household == 0:
        return f'If the population consumes a significant proportion of {row.food_lower} directly in the home, consider expanding the legislation scope to include fortification for household use. This can help increase the amount of fortified vs. unfortified {row.food_lower} consumers are consuming. '
    else: 
        return 'Sin evaluación'

def uses_pf(row):
    if row.status_food == 1 and row.household == 1:
        return f'Se requiere la fortificación para {row.food_lower} que se destina para la fabricación de alimentos procesados. Esto aumentará el acceso y consumo {row.of_food_lower_fortify} y el impacto nutricional.'
    elif row.status_food == 1 and row.household == 0:
        return f'Si una cantidad significante {row.food_lower} se utiliza en alimentos procesados (p. ej., caldo, pan, fideos, etc.), considerar ampliar el alcance de la legislación para incluir la fortificación {row.of_food_lower_use} como ingrediente en la fabricación de alimentos procesados. Esto aumentará el acceso y consumo {row.of_food_lower_fortify} y el impacto nutricional.'
    else:
        return 'Sin evaluación'

def uses_af(row):
    if row.status_food == 1 and row.animal_feed == 1 and row.redcap_event_name == 'salt_arm_2':
        return 'Se requiere la fortificación para alimentación animal, lo cual puede ayudar a aumentar la ingesta de nutrientes en animales y personas a través del consumo de productos de origen animal.'
    elif row.status_food == 1 and row.animal_feed == 0:
        return 'Si una proporción significativa del suministro nacional de sal es utilizada en la alimentación animal, considerar ampliar el alcance de la legislación para incluir el requisito de que la alimentación animal utilice sal fortificada. La sal fortificada es importante para la salud y productividad de los propios animales y también puede ayudar a aumentar la ingesta de nutrientes en humanos a través del consumo de productos de origen animal.'
    else:
        return 'Sin evaluación'

def uses_df(row):
    if row.status_food == 1 and row.donated_food == 1:
        return f'Se requiere la fortificación para {row.food_lower_food} que se dona y programas de alimentación institucional, lo cual puede ayudar a aumentar el número de consumidores que acceden a {row.food_lower}, especialmente, aquellos con ingresos más bajos o aquellos afectados por emergencias.'
    elif row.status_food == 1 and row.donated_food == 0:
        return f'Si se prevé que el país será el principal receptor {row.of_food_lower_donate} (p.ej., a través de programas externos de asistencia alimentaria de emergencia) o utiliza {row.food_lower_food} en programas de alimentación institucional (p.ej., programas de alimentación escolar y redes de protección social), considerar ampliar el alcance de la legislación para incluir el requisito de fortificar {row.food_lower} para donación o de uso en programas de alimentación institucional.'
    else:
        return 'Sin evaluación'


In [ ]:
# Create second dataset with only the result of the analysis
# Applies the respective formulas for each column

final['status_comparison'] = final.apply(lambda row: status_comparison(row), axis=1)
final['standard_comparison'] = final.apply(lambda row: standard_comparison(row), axis=1)
final['import_comparison'] = final.apply(lambda row: import_comparison(row), axis=1)
final['export_comparison'] = final.apply(lambda row: export_comparison(row), axis=1)
final['type_rec'] = final.apply(lambda row: scope_type(row), axis=1)
final['origin_domestically_produced_rec'] = final.apply(lambda row: origin_dp(row), axis=1)
final['origin_import_rec'] = final.apply(lambda row: origin_imports(row), axis=1)
final['origin_exports_rec'] = final.apply(lambda row: origin_exports(row), axis=1)
final['uses_household_rec'] = final.apply(lambda row: uses_household(row), axis=1)
final['uses_processed_food_rec'] = final.apply(lambda row: uses_pf(row), axis=1)
final['uses_animal_feed_rec'] = final.apply(lambda row: uses_af(row), axis=1)
final['uses_donated_food_rec'] = final.apply(lambda row: uses_df(row), axis=1)

# Drop columns that are not the result of the analysis
final.drop(['status_food','fortification_standard', 
            'ext_mon_protocol', 'imp_mon_protocol', 'legislation_scope_types', 'domestically_produced',
            'imports', 'exports', 'household', 'processed_food', 'animal_feed', 'donated_food', 'imp_applicability', 'food_lower', 'food_lower_food', 'of_food_lower', 'of_food_lower_fortify','of_food_lower_use', 'of_food_lower_donate'
            ], axis=1, inplace=True)

In [ ]:
# Formats data into acceptable table for import into REDCap
final.set_index(['country_code', 'redcap_event_name'], inplace=True)

In [ ]:
final

intake_cat  ...                              uses_donated_food_rec
country_code redcap_event_name                                      ...                                                   
1            maize_flour_arm_2         Ingesta/disponibilidad baja  ...                                     Sin evaluación
             oil_arm_2            Ingesta/disponibilidad moderada   ...                                     Sin evaluación
             rice_arm_2                Ingesta/disponibilidad baja  ...                                     Sin evaluación
             salt_arm_2                Ingesta/disponibilidad alta  ...  Si se prevé que el país será el principal rece...
             wheat_flour_arm_2         Ingesta/disponibilidad alta  ...                                     Sin evaluación
...                                                            ...  ...                                                ...
400          maize_flour_arm_2  Ingesta/disponibilidad desconocida  ...                                     Sin evaluación
             oil_arm_2          Ingesta/disponibilidad desconocida  ...                                     Sin evaluación
             rice_arm_2         Ingesta/disponibilidad desconocida  ...                                     Sin evaluación
             salt_arm_2                Ingesta/disponibilidad alta  ...  Si se prevé que el país será el principal rece...
             wheat_flour_arm_2  Ingesta/disponibilidad desconocida  ...  Si se prevé que el país será el principal rece...

[980 rows x 23 columns]

In [ ]:
# FINAL IMPORT - Import to REDCap through API
project.import_records(final)

{'count': 196}